<a href="https://colab.research.google.com/github/sakib762/Machine-Learning-Experiment/blob/main/Air_Quality_Forecasting_ML_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Importing Dependencies








In [ ]:
#importing neccessary staff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Data Collection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/database/ML Project Database/AirQualityUCI.csv')

In [ ]:
df.head()

**All the value in the csv file are separated by ";"**

few column have comma in the place of decimal point


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/database/ML Project Database/AirQualityUCI.csv',sep=';')
df

# Removing Unneccessary Column


In [ ]:
#removing the last two column
df.drop(['Unnamed: 15','Unnamed: 16'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.tail()

# Data Analysis and Processing

In [ ]:
df.shape

Locating Last Data Point

In [ ]:
df.iloc[9356]

In [ ]:
df.iloc[9357]

In [ ]:
#taking first 9356th row for the dataframe
df = df.iloc[:9357,:]

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
#getiing some info of the dataset
df.info()

In [ ]:
#seeing statistical data
df.describe()

In [ ]:
#looking for null or missing in df
df.isnull().sum()

# Missing Value Labeled as "-200"
This shows that there is no missing value in the dataframe, But actual missing values are tagged with the value "-200"

**This is Described in UCI repository documentation**

In [ ]:
#counting the number of times "-200" appears in the data
df.isin([-200]).sum()

# Handling Missing values

Convert every -200 to NaN value

And Replace all NaN values with mean of that specific column

In [ ]:
#converting -200 to NaN value
df.replace(-200,np.nan,inplace=True)

In [ ]:
df.isin([-200]).sum()

In [ ]:
df.isin([np.nan]).sum()

In [ ]:
df.tail()

In [ ]:

df['Time'] = pd.to_datetime(df['Time'], format='%H.%M.%S', errors='coerce').dt.time

In [ ]:
df.head()

In [ ]:
numeric_features = df.select_dtypes(include=np.number)

# Calculate the mean of the numerical features
means = numeric_features.mean()

print(means)

In [ ]:
df = df.fillna(means)

In [ ]:
df

# Two Tasks can be Performed on this Dataset


*   Forecasting
*   Regression



**Forecasting with FB Prophet Algorithm**

In [ ]:
#converting the date to YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

In [ ]:
df

In [ ]:
#creating variable for time info to cnage it
time = df['Time']

In [ ]:
print(time)

In [ ]:
#combining two series to a panda dataframe

date_time = pd.concat([df["Date"],df["Time"]],axis=1)
print(date_time)

In [ ]:
#combining date and time
date_time["ds"] = date_time['Date'].astype(str) + ' ' + date_time['Time'].astype(str)

In [ ]:
date_time

**Changing DS column type to Date-Time**

In [ ]:
data = pd.DataFrame()
#converting ds column to date-time type
data['ds'] = pd.to_datetime(date_time['ds'])

In [ ]:
data

In [ ]:
data["y"] = df["RH"]
data["y"] = pd.to_numeric(data["y"].str.replace(',', '.'))
data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convert 'Time' column to total seconds since midnight
# This represents time as a numeric value
df['Time_Seconds'] = df['Time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

# Drop the original 'Time' column and other non-numeric columns
numeric_df = df.select_dtypes(include=np.number).drop(columns=['Time_Seconds'])  # Exclude 'Time_Seconds' for correlation

# Now calculate and plot the correlation heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(numeric_df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Line plot to visualize trends over time
plt.figure(figsize=(12, 5))
plt.plot(pd.to_datetime(df["Date"]), df["RH"], label="Humidity (RH)")
plt.xlabel("Date")
plt.ylabel("Relative Humidity")
plt.title("Humidity Trend Over Time")
plt.legend()
plt.show()

# Distribution of a selected feature
plt.figure(figsize=(8, 4))
sns.histplot(df["RH"], bins=30, kde=True)
plt.title("Distribution of Humidity Levels")
plt.show()




# Model Implementation

In [ ]:
#installing Prophet model
!pip install prophet

In [ ]:
from prophet import Prophet

In [ ]:
model = Prophet()
model.fit(data)
future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)
model.plot(forecast)
plt.show()
model.plot_components(forecast)
plt.show()

In [ ]:
future = model.make_future_dataframe(periods=365, freq="H")
forecast = model.predict(future)
future.tail()

In [ ]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
#showing plot
fig1 = model.plot(forecast)

In [ ]:
fig2 = model.plot_components(forecast)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Extract actual and predicted values
actual_values = data["y"]  # True values
predicted_values = forecast["yhat"][:len(actual_values)]  # Predicted values

# Compute error metrics
mae = mean_absolute_error(actual_values, predicted_values)
rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
